#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']
print(collection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'exercices'), 'kickstarter')


In [38]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [24]:
df_ks.columns[[6,8,10,12]]
df_ks['country'].

count     150000
unique        24
top           US
freq      115977
Name: country, dtype: object

## Question 0

### Netoyer les données

In [7]:
df_ks.isna().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

### Importer les données

In [45]:
collection.drop()
collection.insert_many(df_ks.to_dict('records'))

In [10]:
database.collection_names()

['kickstarter']

## Question 1  

In [46]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fccf73fdb72f7e4a961c858'),
  'ID': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': '2016-05-10 15:53:56',
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('5fccf73fdb72f7e4a9617736'),
  'ID': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': '2014-10-23 20:06:37',
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('5fccf73edb72f7e4a9614ee6'),
  'ID': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'ma

## Question 2

In [47]:
cur = collection.count({"state":"successful"})
print(cur)

53040


## Question 3

In [48]:
cur = database.kickstarter.aggregate([{"$group" : {"_id" : "$category", "nombre dans categorie" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Comic Books', 'nombre dans categorie': 1032},
 {'_id': 'Workshops', 'nombre dans categorie': 59},
 {'_id': 'Faith', 'nombre dans categorie': 439},
 {'_id': 'Ceramics', 'nombre dans categorie': 128},
 {'_id': 'Quilts', 'nombre dans categorie': 32},
 {'_id': 'Grace is Leaving', 'nombre dans categorie': 1},
 {'_id': 'Apparel', 'nombre dans categorie': 2827},
 {'_id': 'Vegan', 'nombre dans categorie': 229},
 {'_id': 'People', 'nombre dans categorie': 440},
 {'_id': 'Animation', 'nombre dans categorie': 1017},
 {'_id': 'Print', 'nombre dans categorie': 288},
 {'_id': 'Electronic Music', 'nombre dans categorie': 858},
 {'_id': 'Musical', 'nombre dans categorie': 367},
 {'_id': 'Farms', 'nombre dans categorie': 482},
 {'_id': 'Ready-to-wear', 'nombre dans categorie': 332},
 {'_id': 'Immersive', 'nombre dans categorie': 131},
 {'_id': 'Pop', 'nombre dans categorie': 1302},
 {'_id': 'Comedy', 'nombre dans categorie': 923},
 {'_id': 'Narrative Film', 'nombre dans categorie': 2099},
 {'

## Question 4

In [76]:
cur = collection.count({"$and":[{"country":"FR"}, {"launched":{"$ne":2016}}]})
print(cur)

1173


## Question 5

In [43]:
cur = collection.find({"$and":[{"country":"US"}, {"usd_pledged_real":{"$gte" :200000}}]})
#list(cur)

## Question 6 

In [75]:
cur = collection.count({"name": "/Sport/i"})
print(cur)

0
